# 10 Minutes to distributed GPU accelerated XGBoost using dmlc/xgboost's Dask API

The [Dask API of DMLC XGBoost](https://xgboost.readthedocs.io/en/latest/tutorials/dask.html) enables distributed GPU accelerated XGBoost via distributed CUDA DataFrame, Dask-cuDF. A user may pass a reference to a distributed cuDF object, and start a training session over an entire cluster from Python. For a better understanding of this Dask API, please refer to [this article](https://medium.com/rapids-ai/a-new-official-dask-api-for-xgboost-e8b10f3d1eb7)

Let's get started..

### Disable NCCL P2P. Only necessary for versions of NCCL < 2.4

In [1]:
%env NCCL_P2P_DISABLE=1

env: NCCL_P2P_DISABLE=1


### Import necessary modules and initialize the Dask-cuDF Cluster

Using `LocalCUDACluster` from Dask-CUDA to instantiate the single-node cluster.

A user may instantiate a Dask-cuDF cluster like this:

In [3]:
import cudf
import cupy
import dask
import dask_cudf
import xgboost as xgb
import pandas as pd

from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster

import subprocess

cmd = "hostname --all-ip-addresses"
process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
output, error = process.communicate()
IPADDR = str(output.decode()).split()[0]

cluster = LocalCUDACluster(ip=IPADDR)
client = Client(cluster)
client

Client Scheduler: tcp://10.33.227.157:36099 Dashboard: http://10.33.227.157:8787/status,Cluster Workers: 7 Cores: 7 Memory: 540.94 GB


Note the use of `from dask_cuda import LocalCUDACluster`. [Dask-CUDA](https://github.com/rapidsai/dask-cuda) is a lightweight set of utilities useful for setting up a Dask cluster. These calls instantiate a Dask-cuDF cluster in a single node environment. To instantiate a multi-node Dask-cuDF cluster, a user must use `dask-scheduler` and `dask-cuda-worker`. These are utilities available at the command-line to launch the scheduler, and its associated workers.

### Initialize a Random Dataset

Use `dask_cudf.DataFrame.query` to split the dataset into train-and-test sets

In [4]:
size = 1000000
npartitions = 8

cdf = cudf.DataFrame({'x': cupy.random.randint(0, npartitions, size=size), 'y': cupy.random.normal(size=size)})
ddf = dask_cudf.from_cudf(cdf,npartitions=npartitions)

X_train = ddf.query('y < 0.5')
Y_train = X_train[['x']]
X_train = X_train[X_train.columns.difference(['x'])]

X_test = ddf.query('y > 0.5')
Y_test = X_test[['x']]
X_test = X_test[X_test.columns.difference(['x'])]

### Use Dask API for XGBoost

Create a DaskDMatrix object from our training input and target data using `xgb.dask.DaskMatrix`

The DaskDMatrix constructor forces all lazy computation to materialize. To isolate the computation in DaskDMatrix from other lazy computations, one can use wait. Please note: this is optional. For more details, [refer here](https://xgboost.readthedocs.io/en/latest/tutorials/dask.html#why-is-the-initialization-of-daskdmatrix-so-slow-and-throws-weird-errors)

In [5]:
# Here is the XGBoost version used in this example
xgb.__version__

'1.1.1'

In [6]:
## Optional: this wont return until all data is in GPU memory
done = wait([X_train, Y_train])


dtrain = xgb.dask.DaskDMatrix(client, X_train, Y_train)

### Define Parameters and Train with XGBoost

The wall time output below indicates how long it took your GPU cluster to train an XGBoost model over the training set.

Use `dask_cudf.DataFrame.persist()` to ensure each GPU worker has ownership of data before training for optimal load-balance. Please note: this is optional.

In [7]:
%%time

params = {
  'num_rounds':   100,
  'max_depth':    8,
  'max_leaves':   2**8,
  'tree_method':  'gpu_hist',
  'objective':    'reg:squarederror',
  'grow_policy':  'lossguide'
}

## Optional: persist training data into memory
# X_train = X_train.persist()
# Y_train = Y_train.persist()

trained_model = xgb.dask.train(client,params, dtrain, num_boost_round=params['num_rounds'])

CPU times: user 190 ms, sys: 23.2 ms, total: 213 ms
Wall time: 2.4 s


#### Inputs for `xgb.dask.train`

1. `client`: the `dask.distributed.Client`
2. `params`: the training parameters for XGBoost. 
3. `dtrain`: an instance of `xgboost.dask.DaskDMatrix` containing our training input and target data.
4. `num_boost_round=params['num_rounds']`: a specification on the number of boosting rounds for the training session

### Compute Predictions and the RMSE Metric for the Model

This distributed `xgb.dask.train` method returns a dictionary containing the resulting booster and evaluation history obtained from evaluation metrics.

In [8]:
booster = trained_model["booster"] # "Booster" is the trained model
history = trained_model['history'] # "History" is a dictionary containing evaluation results 

booster.set_param({'predictor': 'gpu_predictor'}) #Setting this parameter to run predictions on GPU

Below there are two different methods shown to get the predictions. Using `booster.inplace_predict()` and `xgb.dask.predict`. The inplace_predict can be faster if the prediction is run on the same workers that were used for training. Here are the details from the [doc](https://xgboost.readthedocs.io/en/latest/python/python_api.html#xgboost.Booster.inplace_predict): 

Unlike predict method, inplace prediction does not cache the prediction result.
Calling only inplace_predict in multiple threads is safe and lock free. But the safety does not hold when used in conjunction with other methods. E.g. you can’t train the booster in one thread and perform prediction in the other.

In [9]:
%%time
#Method 1: Using predict
pred = xgb.dask.predict(client,booster, X_test)
true = Y_test['x']
pred = pred.map_partitions(lambda part: cudf.Series(part))
pred = pred.reset_index(drop=True)
true = true.reset_index(drop=True)


squared_error = ((pred - true)**2)
rmse = cupy.sqrt(squared_error.mean().compute())
print('rmse value:', rmse)

/home/nfs/rabhojwani/anaconda3/envs/rapids-env2/lib/python3.7/site-packages/distributed/worker.py:3378: UserWarning: Large object of size 16.23 MB detected in task graph: 
  [<function predict.<locals>.mapped_predict at 0x7f ... titions>, True]
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s)


rmse value: 2.2921026956790693
CPU times: user 277 ms, sys: 265 ms, total: 542 ms
Wall time: 873 ms


NOTE: We mapped each partition of the result from `xgb.dask.predict` into `cudf.Series` to be able to substract it from `true` data. This is now taken care off inside `xgb.dask.predict` itself [[ref]](https://github.com/dmlc/xgboost/pull/5710) and will be available in the next stable version.

#### How to run prediction via `xgb.dask.predict`

1. `client`: the `dask.distributed.Client`
2. `booster`: the Booster produced by the XGBoost training session
3. `X_test`: an instance of `dask_cudf.core.DataFrame` containing the data to be inferenced (acquire predictions)

`pred` and `true` are instances of `dask_cudf.core.Series` object. We align the index of both the Series by resetting it through `reset_index`. To compute the root mean squared error(RMSE) we first compute difference squared between the `pred` and `true` series through the operation `squared_error = ((pred - true)**2)`

Finally, the mean is computed by using an aggregator from the `dask_cudf` API. The entire computation is initiated via `.compute()`. We take the square-root of the result, leaving us with `rmse = cupy.sqrt(squared_error.mean().compute())`.

In [9]:
%%time
#Method 2: Using inplace_predict
pred = booster.inplace_predict(data=X_test.compute())
true = Y_test['x']
true = true.reset_index(drop=True).compute()

squared_error = ((pred - true)**2)
rmse = cupy.sqrt(squared_error.mean())
print('rmse value:', rmse)

rmse value: 2.2899321386939073
CPU times: user 204 ms, sys: 117 ms, total: 321 ms
Wall time: 490 ms


#### How to run prediction via `booster.predict`

1. `data`: This parameter takes these foloowing datatypes: numpy.ndarray/scipy.sparse.csr_matrix/cupy.ndarray/cudf.DataFrame/pd.DataFrame. 

We run `X_test.compute()` to convert `dask_cudf.Dataframe` into `cudf.DataFrame`

Above we can see that `inplace_predict` takes `490 ms` while `predict` takes `873 ms`